In [5]:
# Install bayesian-optimization package if not already installed
# !pip install bayesian-optimization

from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

In [6]:
def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    """Random Forest cross validation.
    This function will instantiate a random forest classifier with
    parameters n_estimators, min_samples_split, and max_features.
    Combined with data and targets this will be used to perform cross-validation.
    The result of cross-validation is returned. Our goal is to find
    combinations of n_estimators, min_samples_split, and max_features that
    maximizes the mean cross-validation score.
    """
    estimator = RFC(
        n_estimators=int(n_estimators),
        min_samples_split=int(min_samples_split),
        max_features=max(min(max_features, 0.999), 1e-3),
        random_state=2
    )
    cval = cross_val_score(estimator, data, targets, scoring='accuracy', cv=4)
    return cval.mean()

In [7]:
def optimize_rfc(data, targets):
    """Apply Bayesian Optimization to Random Forest parameters."""
    def rfc_crossval(n_estimators, min_samples_split, max_features):
        """Wrapper of RandomForest cross-validation.
        Ensure n_estimators and min_samples_split are cast to integers.
        Ensure max_features is within (0, 1) range.
        """
        return rfc_cv(
            n_estimators=n_estimators,
            min_samples_split=min_samples_split,
            max_features=max_features,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            "n_estimators": (10, 250),
            "min_samples_split": (2, 25),
            "max_features": (0.1, 0.999),
        },
        random_state=1234,
        verbose=2
    )
    
    optimizer.maximize(n_iter=10)
    print("Final result:", optimizer.max)
    return optimizer

In [8]:
# Set seed for reproducibility
seed = 2017
np.random.seed(seed)

In [9]:
# Read the data in
df = pd.read_csv("diabetes.csv")
X = df.iloc[:, :8].values  # Independent variables
y = df['Outcome'].values  # Dependent variables

In [10]:
# Normalize
X = StandardScaler().fit_transform(X)

In [11]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

In [12]:
# Perform Bayesian Optimization
print("--- Optimizing Random Forest ---")
optimize_rfc(X_train, y_train)

--- Optimizing Random Forest ---
|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.7486    | 0.2722    | 16.31     | 115.1     |
| 2         | 0.7467    | 0.806     | 19.94     | 75.42     |
| 3         | 0.7448    | 0.3485    | 20.44     | 240.0     |
| 4         | 0.7579    | 0.8875    | 10.23     | 130.2     |
| 5         | 0.7486    | 0.7144    | 18.39     | 98.86     |
| 6         | 0.7504    | 0.4713    | 23.55     | 96.06     |
| 7         | 0.7467    | 0.638     | 6.223     | 160.9     |
| 8         | 0.7486    | 0.5152    | 9.149     | 139.0     |
| 9         | 0.7504    | 0.9029    | 18.17     | 241.5     |
| 10        | 0.7579    | 0.8899    | 10.23     | 130.2     |
| 11        | 0.7504    | 0.999     | 11.97     | 128.4     |
| 12        | 0.7467    | 0.8317    | 9.111     | 131.4     |
| 13        | 0.756     | 0.9109    | 10.7      | 129.7     |
| 14        | 0.7485    | 0.1359    |